In [1]:
%run ../yp_utils.py

# Initial setup

In [5]:
paper_pmid = 35298616
paper_name = 'arras_ganley_2022' 

In [6]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [7]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_excel('raw_data/Supplementary_Table_3.xlsx', sheet_name='Sort_ORF')

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 107 x 3


In [10]:
original_data.head()

,ORF,Gene Name,Description
0,YAL013W,DEP1,Transcriptional modulator involved in regulati...
1,YAL054C,ACS1,"Acetyl-coA synthetase isoform which, along wit..."
2,YBR028C,YPK3,An AGC kinase phosphorylated by cAMP-dependent...
3,YBR032W,YBR032W,Dubious open reading frame unlikely to encode ...
4,YBR121C,GRS1,Cytoplasmic and mitochondrial glycyl-tRNA synt...


In [11]:
original_data['orf'] = original_data['ORF'].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF, Gene Name, Description, orf]
Index: []


In [15]:
original_data['data'] = -1

In [16]:
original_data.set_index('orf', inplace=True)

In [17]:
original_data = original_data[['data']].copy()

In [18]:
original_data = original_data.groupby(original_data.index).mean()

In [19]:
original_data.shape

(106, 1)

# Load & process tested strains

In [20]:
tested = pd.read_excel('raw_data/Boone Array Genes.xlsx', sheet_name='Boone Array (384)', skiprows=1)

In [21]:
tested.head()

,Notes,ORF,Gene,Updated Name,Plate,Row,Column,Notes.1,Unnamed: 8
0,NaN,YOR202W,HIS3,HIS3,1,1,1,NaN,green=often SL with wild-type
1,NaN,YOR202W,HIS3,HIS3,1,1,2,NaN,blue=expected to appear SL
2,NaN,YOR202W,HIS3,HIS3,1,1,3,NaN,NaN
3,NaN,YOR202W,HIS3,HIS3,1,1,4,NaN,NaN
4,NaN,YOR202W,HIS3,HIS3,1,1,5,NaN,NaN


In [23]:
tested.columns

Index(['Notes', 'ORF ', 'Gene', 'Updated Name', 'Plate', 'Row', 'Column',
       'Notes.1', 'Unnamed: 8'],
      dtype='object')

In [24]:
tested['orf'] = tested['ORF '].astype(str)

In [25]:
tested['orf'] = clean_orf(tested['orf'])

In [26]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [27]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

Empty DataFrame
Columns: [Notes, ORF , Gene, Updated Name, Plate, Row, Column, Notes.1, Unnamed: 8, orf]
Index: []


In [28]:
tested_orfs = tested['orf'].unique()

In [29]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

[]

In [30]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

# Prepare the final dataset

In [31]:
data = original_data.copy()

In [32]:
dataset_ids = [22219]
datasets = datasets.reindex(index=dataset_ids)

In [33]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [34]:
data.head()

dataset_id,22219
data_type,value
orf,
YOR202W,-1
YBR030W,0
YBR138C,0
YBR028C,-1
YBR137W,0


## Subset to the genes currently in SGD

In [35]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [36]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,22219
,data_type,value
gene_id,orf,
5414,YOR202W,-1
226,YBR030W,0
332,YBR138C,0
224,YBR028C,-1
331,YBR137W,0


# Normalize

In [37]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [38]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [39]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id      22219          
data_type       value    valuez
gene_id orf                    
5414    YOR202W    -1 -6.329029
226     YBR030W     0  0.000000
332     YBR138C     0  0.000000
224     YBR028C    -1 -6.329029
331     YBR137W     0  0.000000

# Print out

In [40]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [41]:
from IO.save_data_to_db3 import *

In [42]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 35298616...
Inserting the new data...


100%|██████████| 1/1 [00:08<00:00,  8.32s/it]

Updating the data_modified_on field...
